# Overview
This notebook will help to provide some documentation and reproducibility to training a username splitter.

All necessary files are in the py-bkstring repo on the "research" branch.

## Dependencies
You will need to install the following python library:
* [sklearn-crfsuite](https://github.com/TeamHG-Memex/sklearn-crfsuite)

## Problem
The main problem with reference to username matching, is that usernames are often made up of one or more words.  The main problem is that these words are not always separated by any discernible delimeter.

## Suggested Solution
In order to split names effectively, we will train a model to recognize where usernames should be broken apart.  The break points will be determined by character, so we know at the very least we should be gathering features based on each character within the 

To do this we will:

1. Import all the things!
1. Clean the dataset
1. Process features
1. Make the training set
1. Train the model
1. Show the results

## Import all the things!

In [39]:
import re
import json

from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics

## Clean the dataset
Our dataset comes from a list of matching names from the "data/name-matches.txt" file, which takes the format:

```
=========================
<name1>
<name2>
<name3>
=========================
<nameA>
...
```

Where all names within the equals marks are a matching set.  This isn't particularly useful for name splitting, so we'll just create a list which will have the username, and the index of all spaces.

In [40]:
def names():
    with open('data/name-matches.txt', 'r') as f:
        data = f.read()

    data = data.split('\n=====================================\n')

    ret = list()
    for line in data:
        for name in line.split('\n'):
            if ' ' not in name:
                continue

            regex = [m.start() for m in re.finditer('\s', name)]
            ret.append((name, regex))

    return ret

# Take a look at the first name in the list as well as the whitespace index for that name.
print(names()[0])

('kalsen 2013', [6])


_Note: 'kalsen 2013' is the username, and there is a space at index of 6._

## Process features
We now need to figure out our mechanism to retrieve features from a character at a position in a given username.  A good starting point would be tracking the following data from each character:

* lower (Character)
* isupper (Boolean)
* isdigit (Boolean)
* islower (Boolean)
* isalpha (Boolean)
* position (Number)

It is also helpful to know the same information about characters 1-3 positions ahead of and behind each given character.  This allows us train patterns of nearby characters as well.

In [41]:
# This function has a lot of redundant code and will be cleaned up in actual implementation.
def features(name, i, flag=None):
    char = name[i]

    features = {
        'lower': char.lower(),
        'isupper': char.isupper(),
        'isdigit': char.isdigit(),
        'islower': char.islower(),
        'isalpha': char.isalpha(),
        'position': i,
        'length': len(name)
    }

    if i == 0:
        features['BON'] = True
    else:
        char = name[i - 1]
        features.update({
            '-1:lower': char.lower(),
            '-1:isupper': char.isupper(),
            '-1:isdigit': char.isdigit(),
            '-1:islower': char.islower(),
            '-1:isalpha': char.isalpha(),
            '-1:position': i - 1
        })

    if i > 1:
        char = name[i - 2]
        features.update({
            '-2:lower': char.lower(),
            '-2:isupper': char.isupper(),
            '-2:isdigit': char.isdigit(),
            '-2:islower': char.islower(),
            '-2:isalpha': char.isalpha(),
            '-2:position': i - 2
        })

    if i > 2:
        char = name[i - 3]
        features.update({
            '-3:lower': char.lower(),
            '-3:isupper': char.isupper(),
            '-3:isdigit': char.isdigit(),
            '-3:islower': char.islower(),
            '-3:isalpha': char.isalpha(),
            '-3:position': i - 3
        })

    if i == len(name) - 1:
        features['EON'] = True
    else:
        char = name[i + 1]
        features.update({
            '+1:lower': char.lower(),
            '+1:isupper': char.isupper(),
            '+1:isdigit': char.isdigit(),
            '+1:islower': char.islower(),
            '+1:isalpha': char.isalpha(),
            '+1:position': i + 1
        })

    if i < len(name) - 2:
        char = name[i + 2]
        features.update({
            '+2:lower': char.lower(),
            '+2:isupper': char.isupper(),
            '+2:isdigit': char.isdigit(),
            '+2:islower': char.islower(),
            '+2:isalpha': char.isalpha(),
            '+2:position': i + 2
        })

    if i < len(name) - 3:
        char = name[i + 3]
        features.update({
            '+3:lower': char.lower(),
            '+3:isupper': char.isupper(),
            '+3:isdigit': char.isdigit(),
            '+3:islower': char.islower(),
            '+3:isalpha': char.isalpha(),
            '+3:position': i + 3
        })

    return features

# Let's take a look at the feature dictionary!
print(json.dumps(features('johndoe1', 1), indent=2))

{
  "+2:isupper": false,
  "+3:isdigit": false,
  "+3:position": 4,
  "+3:isupper": false,
  "+2:isdigit": false,
  "-1:position": 0,
  "+2:lower": "n",
  "+1:position": 2,
  "position": 1,
  "isdigit": false,
  "+3:lower": "d",
  "isupper": false,
  "+1:isalpha": true,
  "+1:lower": "h",
  "-1:lower": "j",
  "length": 8,
  "-1:isdigit": false,
  "-1:isalpha": true,
  "+2:isalpha": true,
  "isalpha": true,
  "islower": true,
  "+1:islower": true,
  "+2:islower": true,
  "-1:isupper": false,
  "-1:islower": true,
  "+1:isdigit": false,
  "+1:isupper": false,
  "lower": "o",
  "+3:isalpha": true,
  "+3:islower": true,
  "+2:position": 3
}


## Make the training set
We will need a set of features for a bunch of names, as well as set of whether there should be a breakpoint after the given character.  Our training data will be created by processing usernames with spaces, and putting a marker where the spaces exist, and then taking the spaces out before handing it over to the modeller.

Example:
```
original_name = 'john doe'
training_name = 'johndoe'
spaces = [ 4 ]

```

At `name[0]` through `name[2]`, and `name[4]` through `name[6]` we will label as 'no_split_next,' and on `name[3]` we will label as 'yes_split_next.'

This allows us to attempt to find breakpoints in names, with a pretty solid grounding truth of actually containing a space.  But the training set will have to work extra hard to try to find likely breakpoints in a username without being given an easy and obvious delimeter.

In [42]:
def name_features(word, spaces):
    word = word.replace(' ', '')
    flag = None
    feature_list = list()
    split_list = list()

    i_spacer = 1

    for i in range(len(word)):
        split_flag = 'no_split_next'

        if i + i_spacer in spaces:
            split_flag = 'yes_split_next'
            i_spacer += 1

        feature_list.append(features(word, i, flag=flag))
        split_list.append(split_flag)

    return (feature_list, split_list)

# This is just to show what labels look like for each letter in the username.
feat, split = name_features('john doe', list([4]))
for i in range(len(feat)):
    print(feat[i]['lower'], split[i])

j no_split_next
o no_split_next
h no_split_next
n yes_split_next
d no_split_next
o no_split_next
e no_split_next


In [43]:
# Now we'll actually create the training/testing sets.
features = [name_features(name, spaces) for name, spaces in names()]

X_train = list()
Y_train = list()

train_range = range(0, 2500)
for i in train_range:
    x, y = features[i]
    X_train.append(x)
    Y_train.append(y)

test_range = range(2500, 3000)

X_test = list()
Y_test = list()

for i in test_range:
    x, y = features[i]
    X_test.append(x)
    Y_test.append(y)

## Train the model
This sounds super interesting, but it's fairly simple:

In [44]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, Y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

## Show the results
We'll now take a look at our F1 score, though really we're very interested in precision here, and not terribly worried about recall.

In [45]:
labels = list(crf.classes_)

y_pred = crf.predict(X_test)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print(metrics.flat_classification_report(Y_test, y_pred, labels=sorted_labels, digits=3))

             precision    recall  f1-score   support

yes_split_next      0.803     0.687     0.740       550
no_split_next      0.964     0.980     0.972      4653

avg / total      0.947     0.949     0.947      5203



A precision of .80 is really good, considering words in usernames don't have to follow any reason, including normal grammar, spelling, or naming conventions.  We could probably get a higher precision if we had more sources of usernames, and potentially attempt to fit against regular text with spaces in it.

We could also try adding features to slide windows of characters near the current position.  Windows could consist of 2-4 character substrings, and go from -3 to +3.

# Conclusion
This seems like a pretty good way to attempt to split usernames for matching, but we can iterate and attempt to break at other markers other than spaces too.  One concern is that the training data only matches well against the given username set, so we may need to retrain every once in a while when the precision has dropped too low upon adding other usernames.

In the end it's worth attempting to implement, and seeing if it improves our name matching overall.

# Thank You
A big thank you to [kmike](https://github.com/kmike), a lot of this notebook used examples from the [CoNLL2002 Notebook](https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb) from [sklearn-crfsuite](https://github.com/TeamHG-Memex/sklearn-crfsuite).